In [ ]:
import os
import pandas as pd
from pyvis.network import Network

In [ ]:
kills = pd.DataFrame(columns=['universe', 'culprit', 'count', 'victims'])
deaths = pd.DataFrame(columns=['universe', 'victim', 'culprits', 'observer dependent', 'definitive'])

In [ ]:
for path, subdirs, files in os.walk('Lists'):
    for name in files:
        file = path + '\\' + name
        universe = file.split('\\')[1::2][0]
        
        with open(file, 'r') as f:
            lines = f.read().split('\n')
        
        for line in lines:
            victim = line.split('**')[1::2][0]
            culprits = line.split('_')[1::2]

            if '(observer dependent)' in line:
                observer = True
            else:
                observer = False
            
            if '(later revived)' in line or '(later resurrected)' in line or observer:
                definitive = False
            else:
                definitive = True

            for i in range(len(culprits)):
                if culprits[i] == 'suicide':
                    culprits[i] = victim

                row = [universe, culprits[i], 1, victim]
                kills.loc[len(kills)] = row

            row = [universe, victim, culprits, observer, definitive]
            deaths.loc[len(deaths)] = row

In [ ]:
edge_list = kills[['universe', 'culprit', 'victims']]
kills = kills.groupby(by=['universe', 'culprit']).aggregate({'count':sum, 'victims':', '.join})
kills = kills.reset_index().sort_values(by='count', ascending=False)
kills.drop_duplicates('universe')

In [ ]:
sources, targets = edge_list['culprit'], edge_list['victims']

net = Network(height="50vw", width="100%", bgcolor="#111111", font_color="#e8ac87", directed=True)
edge_data = zip(sources, targets)

for src, dst in edge_data:
    net.add_node(src, label=src, color='crimson')
    net.add_node(dst, label=dst, color='crimson')
    net.add_edge(src, dst, color='#eb73b7')

    neighbor_map = net.get_adj_list()

    for node in net.nodes:
        node['title'] = node['label'] + "'s Victims :\n\n" + "\n".join(neighbor_map[node['id']])
        node['value'] = len(neighbor_map[node['id']])
    
    net.inherit_edge_colors(False)
    net.set_edge_smooth('dynamic')

net.save_graph('main.html')